In [1]:
import pandas as pd 

data_dir = '../data/'

animation_df = pd.read_csv(data_dir + 'kuka_framedata.csv')
animation_df.head()

,Frame,Link,Parent,X_Position,Y_Position,Z_Position,X_Rotation,Y_Rotation,Z_Rotation,Y_Velocity,X_Velocity,Z_Velocity
0,1,LinkN0,Robot KUKA LBR Iiwa14,0.000000e+00,0.000000e+00,0.000000,0,0.0,0.0,0.0,0,0
1,1,LinkN0-Axis,LinkN0,-3.490000e-08,0.000000e+00,0.157500,0,0.0,0.0,0.0,0,0
2,1,LinkN1,LinkN0,-3.490000e-08,0.000000e+00,0.157500,0,0.0,0.0,0.0,0,0
3,1,LinkN1-Axis,LinkN1,-1.550000e-09,4.270000e-08,0.360067,0,0.0,0.0,0.0,0,0
4,1,LinkN2,LinkN1,-1.550000e-09,-7.720000e-05,0.360067,0,0.0,0.0,0.0,0,0


In [2]:
print(f"frame df length: {len(animation_df):}")

frame df length: 1425


In [3]:
filtered_df = animation_df[~animation_df['Link'].str.contains('Axis')]
filtered_df.head()
print(f"filtered df length: {len(filtered_df):}")


filtered df length: 855


In [4]:
links = ["LinkN0", "LinkN1", "LinkN2", "LinkN3", "LinkN4", "LinkN5", "LinkN6", "LinkN7"]
weird_data = filtered_df[~filtered_df['Link'].isin(links)]
print(f"link names: {weird_data['Link'].unique()}")

link names: ['LinkN6.002']


In [5]:
weird_link = "LinkN6.002"
print(f"number of rows containing {weird_link}: {len(filtered_df[filtered_df['Link'] == weird_link])}")

number of rows containing LinkN6.002: 95


In [6]:
cleaned_df = filtered_df[~filtered_df['Link'].str.contains(weird_link)]
print(len(cleaned_df))

760


In [7]:
print(f"unique link names before cleaning: {animation_df['Link'].unique()}")
print(f"unique link names after cleaning: {cleaned_df['Link'].unique()}")


unique link names before cleaning: ['LinkN0' 'LinkN0-Axis' 'LinkN1' 'LinkN1-Axis' 'LinkN2' 'LinkN2-Axis'
 'LinkN3' 'LinkN3_Axis' 'LinkN4' 'LinkN4_Axis' 'LinkN5' 'LinkN5_Axis'
 'LinkN6' 'LinkN6.002' 'LinkN7']
unique link names after cleaning: ['LinkN0' 'LinkN1' 'LinkN2' 'LinkN3' 'LinkN4' 'LinkN5' 'LinkN6' 'LinkN7']


In [8]:
link_dfs = {}

for link in links:
    link_dfs[link] = cleaned_df[cleaned_df['Link'] == link].reset_index(drop=True)
    if link != "LinkN7":
        link_dfs[link]['qpos'] = link_dfs[link]['Y_Rotation']
        link_dfs[link]['qvel'] = link_dfs[link]['Y_Velocity']
    else:
        link_dfs[link]['qpos'] = link_dfs[link]['Z_Rotation'] 
        link_dfs[link]['qvel'] = link_dfs[link]['Z_Velocity']
        
    print(f"Created dataframe for {link} with {len(link_dfs[link])} rows")



Created dataframe for LinkN0 with 95 rows
Created dataframe for LinkN1 with 95 rows
Created dataframe for LinkN2 with 95 rows
Created dataframe for LinkN3 with 95 rows
Created dataframe for LinkN4 with 95 rows
Created dataframe for LinkN5 with 95 rows
Created dataframe for LinkN6 with 95 rows
Created dataframe for LinkN7 with 95 rows


In [9]:
link_dfs['LinkN6']

,Frame,Link,Parent,X_Position,Y_Position,Z_Position,X_Rotation,Y_Rotation,Z_Rotation,Y_Velocity,X_Velocity,Z_Velocity,qpos,qvel
0,1,LinkN6,LinkN5,-4.440000e-17,-0.06265,1.18,0,0.00000,0.0,0.0,0,0,0.00000,0.0
1,2,LinkN6,LinkN5,-4.440000e-17,-0.06265,1.18,0,0.00000,0.0,0.0,0,0,0.00000,0.0
2,3,LinkN6,LinkN5,-4.440000e-17,-0.06265,1.18,0,0.00000,0.0,0.0,0,0,0.00000,0.0
3,4,LinkN6,LinkN5,-4.440000e-17,-0.06265,1.18,0,0.00000,0.0,0.0,0,0,0.00000,0.0
4,5,LinkN6,LinkN5,-4.440000e-17,-0.06265,1.18,0,0.00000,0.0,0.0,0,0,0.00000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90,91,LinkN6,LinkN5,-4.440000e-17,-0.06265,1.18,0,0.06418,0.0,0.0,0,0,0.06418,0.0
91,92,LinkN6,LinkN5,-4.440000e-17,-0.06265,1.18,0,0.06418,0.0,0.0,0,0,0.06418,0.0
92,93,LinkN6,LinkN5,-4.440000e-17,-0.06265,1.18,0,0.06418,0.0,0.0,0,0,0.06418,0.0
93,94,LinkN6,LinkN5,-4.440000e-17,-0.06265,1.18,0,0.06418,0.0,0.0,0,0,0.06418,0.0


In [10]:
# Start with the first link's frame, qpos and qvel
merged_df = link_dfs[links[0]][['Frame', 'qpos', 'qvel']].copy()
merged_df.rename(columns={
    'qpos': f'qpos_{links[0]}',
    'qvel': f'qvel_{links[0]}'
}, inplace=True)

# Merge with remaining links
for link in links[1:]:
    link_data = link_dfs[link][['Frame', 'qpos', 'qvel']].copy()
    link_data.rename(columns={
        'qpos': f'qpos_{link}',
        'qvel': f'qvel_{link}'
    }, inplace=True)
    merged_df = pd.merge(merged_df, link_data, on='Frame')

In [11]:
print(merged_df.head())

   Frame  qpos_LinkN0  qvel_LinkN0  qpos_LinkN1  qvel_LinkN1  qpos_LinkN2  \
0      1          0.0          0.0          0.0          0.0     0.000000   
1      2          0.0          0.0          0.0          0.0    -0.008752   
2      3          0.0          0.0          0.0          0.0    -0.033218   
3      4          0.0          0.0          0.0          0.0    -0.070715   
4      5          0.0          0.0          0.0          0.0    -0.118558   

   qvel_LinkN2  qpos_LinkN3  qvel_LinkN3  qpos_LinkN4  qvel_LinkN4  \
0     0.000000          0.0          0.0          0.0          0.0   
1    -0.210044          0.0          0.0          0.0          0.0   
2    -0.587190          0.0          0.0          0.0          0.0   
3    -0.899923          0.0          0.0          0.0          0.0   
4    -1.148244          0.0          0.0          0.0          0.0   

   qpos_LinkN5  qvel_LinkN5  qpos_LinkN6  qvel_LinkN6  qpos_LinkN7  \
0          0.0          0.0          0.0      

In [12]:
print(len(merged_df))

95


In [13]:
merged_df.to_csv(data_dir + 'kuka_formatted.csv', index=False)